In [1]:
# encoding=utf8

# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


import pickle

from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Set de entrenamiento con el target Damage Grade includio
data_train = pd.read_csv('data/clean_data.csv',dtype = {
    "building_id" : "int32", #0
    "geo_level_1_id" : "int8", #1
    "geo_level_2_id" : "int16", #2
    "geo_level_3_id" : "int16", #3
    "count_floors_pre_eq" : "int8", #4
    "age" : "int16", #5
    "area_percentage" :"int8", #6
    "height_percentage" :"int8", #7
    "land_surface_condition" :"category", #8
    "foundation_type" :"category", #9
    "roof_type" :"category", #10
    "ground_floor_type" :"category",#11
    "other_floor_type" :"category",#12
    "position" :"category",#13
    "plan_configuration" :"category", #14
    "has_superstructure_adobe_mud" : "bool",
    "has_superstructure_mud_mortar_stone" : "bool",
    "has_superstructure_stone_flag" : "bool",
    "has_superstructure_cement_mortar_stone" : "bool",
    "has_superstructure_mud_mortar_brick" : "bool",
    "has_superstructure_cement_mortar_brick" : "bool",
    "has_superstructure_timber" : "bool",
    "has_superstructure_bamboo" : "bool",
    "has_superstructure_rc_non_engineered" : "bool",
    "has_superstructure_rc_engineered" : "bool",
    "has_superstructure_other" : "bool",
    "legal_ownership_status" :"category",# 26
    "count_families" : "int8",
    "has_secondary_use" : "bool",
    "has_secondary_use_agriculture" : "bool",
    "has_secondary_use_hotel" : "bool",
    "has_secondary_use_rental" : "bool",
    "has_secondary_use_institution" : "bool",
    "has_secondary_use_school" : "bool",
    "has_secondary_use_industry" : "bool",
    "has_secondary_use_health_post" : "bool",
    "has_secondary_use_gov_office" : "bool",
    "has_secondary_use_use_police" : "bool",
    "has_secondary_use_other" : "bool",
    "damage_grade":"int8" })
data_train

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,False,False,False,False,False,False,False,False,False,3
1,28830,8,900,2812,2,10,8,7,o,r,...,False,False,False,False,False,False,False,False,False,2
2,94947,21,363,8973,2,10,5,5,t,r,...,False,False,False,False,False,False,False,False,False,3
3,590882,22,418,10694,2,10,6,5,t,r,...,False,False,False,False,False,False,False,False,False,2
4,201944,11,131,1488,3,30,8,9,t,r,...,False,False,False,False,False,False,False,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,False,False,False,False,False,False,False,False,False,2
260597,669485,17,715,2060,2,0,6,5,t,r,...,False,False,False,False,False,False,False,False,False,3
260598,602512,17,51,8163,3,55,6,7,t,r,...,False,False,False,False,False,False,False,False,False,3
260599,151409,26,39,1851,2,10,14,6,t,r,...,False,False,False,False,False,False,False,False,False,2


In [3]:
#Set de Test
data_test = pd.read_csv('data/test_values.csv',dtype = {
    "building_id" : "int32",
    "geo_level_1_id" : "int8", 
    "geo_level_2_id" : "int16",
    "geo_level_3_id" : "int16",
    "count_floors_pre_eq" : "int8",
    "age" : "int16",
    "area_percentage" :"int8",
    "height_percentage" :"int8",
    "land_surface_condition" :"category",
    "foundation_type" :"category",
    "roof_type" :"category",
    "ground_floor_type" :"category",
    "other_floor_type" :"category",
    "position" :"category",
    "plan_configuration" :"category",
    "has_superstructure_adobe_mud" : "bool",
    "has_superstructure_mud_mortar_stone" : "bool",
    "has_superstructure_stone_flag" : "bool",
    "has_superstructure_cement_mortar_stone" : "bool",
    "has_superstructure_mud_mortar_brick" : "bool",
    "has_superstructure_cement_mortar_brick" : "bool",
    "has_superstructure_timber" : "bool",
    "has_superstructure_bamboo" : "bool",
    "has_superstructure_rc_non_engineered" : "bool",
    "has_superstructure_rc_engineered" : "bool",
    "has_superstructure_other" : "bool",
    "legal_ownership_status" :"category",
    "count_families" : "int8",
    "has_secondary_use" : "bool",
    "has_secondary_use_agriculture" : "bool",
    "has_secondary_use_hotel" : "bool",
    "has_secondary_use_rental" : "bool",
    "has_secondary_use_institution" : "bool",
    "has_secondary_use_school" : "bool",
    "has_secondary_use_industry" : "bool",
    "has_secondary_use_health_post" : "bool",
    "has_secondary_use_gov_office" : "bool",
    "has_secondary_use_use_police" : "bool",
    "has_secondary_use_other" : "bool"})

In [4]:
data_train["has_secondary_use"].value_counts()

False    231445
True      29156
Name: has_secondary_use, dtype: int64

In [5]:
data_train["geo_level_absolute"] = data_train["geo_level_1_id"] + data_train["geo_level_2_id"] + data_train["geo_level_3_id"]
data_test["geo_level_absolute"] = data_test["geo_level_1_id"] + data_test["geo_level_2_id"] + data_test["geo_level_3_id"]

In [6]:
y = data_train["damage_grade"].values
X = data_train.drop(['damage_grade', 'building_id'], axis=1)
X_test = data_test.drop(['building_id'], axis=1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=1, stratify=y, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

((234540, 39), (26061, 39), (234540,), (26061,), (86868, 39))

In [8]:
cate_features_index = np.where(X.dtypes != float)[0]

In [9]:
from catboost import Pool, CatBoostClassifier, cv
model = CatBoostClassifier(eval_metric='Accuracy', loss_function='MultiClass',use_best_model=True,random_seed=1)
model.fit(X_train,y_train,cat_features=cate_features_index,eval_set=(X_val,y_val))
print(model.score(X_train, y_train))

Learning rate set to 0.120429
0:	learn: 0.6971902	test: 0.6979778	best: 0.6979778 (0)	total: 3.87s	remaining: 1h 4m 21s
1:	learn: 0.6970282	test: 0.6976708	best: 0.6979778 (0)	total: 7.87s	remaining: 1h 5m 28s
2:	learn: 0.6978724	test: 0.6982464	best: 0.6982464 (2)	total: 13s	remaining: 1h 12m 12s
3:	learn: 0.6982690	test: 0.6987453	best: 0.6987453 (3)	total: 17.3s	remaining: 1h 11m 42s
4:	learn: 0.7008144	test: 0.7012778	best: 0.7012778 (4)	total: 22.2s	remaining: 1h 13m 29s
5:	learn: 0.7014113	test: 0.7017382	best: 0.7017382 (5)	total: 26s	remaining: 1h 11m 53s
6:	learn: 0.7015776	test: 0.7018150	best: 0.7018150 (6)	total: 30s	remaining: 1h 11m
7:	learn: 0.7035047	test: 0.7031580	best: 0.7031580 (7)	total: 35s	remaining: 1h 12m 17s
8:	learn: 0.7079773	test: 0.7081463	best: 0.7081463 (8)	total: 38.4s	remaining: 1h 10m 27s
9:	learn: 0.7123774	test: 0.7131346	best: 0.7131346 (9)	total: 41.8s	remaining: 1h 9m
10:	learn: 0.7146755	test: 0.7160892	best: 0.7160892 (10)	total: 45.9s	remainin

88:	learn: 0.7372687	test: 0.7456352	best: 0.7458271 (82)	total: 9m 58s	remaining: 1h 42m 6s
89:	learn: 0.7373284	test: 0.7457887	best: 0.7458271 (82)	total: 10m 8s	remaining: 1h 42m 30s
90:	learn: 0.7374563	test: 0.7456736	best: 0.7458271 (82)	total: 10m 19s	remaining: 1h 43m 10s
91:	learn: 0.7374734	test: 0.7456352	best: 0.7458271 (82)	total: 10m 28s	remaining: 1h 43m 27s
92:	learn: 0.7375501	test: 0.7459422	best: 0.7459422 (92)	total: 10m 38s	remaining: 1h 43m 50s
93:	learn: 0.7376482	test: 0.7460957	best: 0.7460957 (93)	total: 10m 46s	remaining: 1h 43m 49s
94:	learn: 0.7377164	test: 0.7457120	best: 0.7460957 (93)	total: 10m 51s	remaining: 1h 43m 27s
95:	learn: 0.7377292	test: 0.7457887	best: 0.7460957 (93)	total: 10m 57s	remaining: 1h 43m 11s
96:	learn: 0.7378741	test: 0.7454050	best: 0.7460957 (93)	total: 11m 3s	remaining: 1h 42m 56s
97:	learn: 0.7379168	test: 0.7456352	best: 0.7460957 (93)	total: 11m 8s	remaining: 1h 42m 37s
98:	learn: 0.7381342	test: 0.7459422	best: 0.7460957 (9

174:	learn: 0.7421634	test: 0.7470166	best: 0.7479759 (141)	total: 18m 28s	remaining: 1h 27m 5s
175:	learn: 0.7422145	test: 0.7471317	best: 0.7479759 (141)	total: 18m 33s	remaining: 1h 26m 51s
176:	learn: 0.7422529	test: 0.7471317	best: 0.7479759 (141)	total: 18m 37s	remaining: 1h 26m 35s
177:	learn: 0.7422401	test: 0.7471701	best: 0.7479759 (141)	total: 18m 42s	remaining: 1h 26m 23s
178:	learn: 0.7422572	test: 0.7473236	best: 0.7479759 (141)	total: 18m 47s	remaining: 1h 26m 9s
179:	learn: 0.7422487	test: 0.7475538	best: 0.7479759 (141)	total: 18m 51s	remaining: 1h 25m 54s
180:	learn: 0.7423126	test: 0.7474387	best: 0.7479759 (141)	total: 18m 56s	remaining: 1h 25m 40s
181:	learn: 0.7422785	test: 0.7478224	best: 0.7479759 (141)	total: 19m	remaining: 1h 25m 27s
182:	learn: 0.7424064	test: 0.7476689	best: 0.7479759 (141)	total: 19m 5s	remaining: 1h 25m 13s
183:	learn: 0.7424149	test: 0.7478608	best: 0.7479759 (141)	total: 19m 10s	remaining: 1h 25m 1s
184:	learn: 0.7424363	test: 0.7480526	

259:	learn: 0.7450158	test: 0.7499712	best: 0.7502015 (247)	total: 27m 10s	remaining: 1h 17m 20s
260:	learn: 0.7450541	test: 0.7500096	best: 0.7502015 (247)	total: 27m 16s	remaining: 1h 17m 12s
261:	learn: 0.7450925	test: 0.7499712	best: 0.7502015 (247)	total: 27m 22s	remaining: 1h 17m 7s
262:	learn: 0.7450968	test: 0.7500863	best: 0.7502015 (247)	total: 27m 29s	remaining: 1h 17m 1s
263:	learn: 0.7450584	test: 0.7499328	best: 0.7502015 (247)	total: 27m 35s	remaining: 1h 16m 54s
264:	learn: 0.7451010	test: 0.7500863	best: 0.7502015 (247)	total: 27m 41s	remaining: 1h 16m 48s
265:	learn: 0.7451735	test: 0.7500480	best: 0.7502015 (247)	total: 27m 47s	remaining: 1h 16m 42s
266:	learn: 0.7451693	test: 0.7499712	best: 0.7502015 (247)	total: 27m 53s	remaining: 1h 16m 34s
267:	learn: 0.7452034	test: 0.7499712	best: 0.7502015 (247)	total: 28m	remaining: 1h 16m 29s
268:	learn: 0.7452076	test: 0.7500480	best: 0.7502015 (247)	total: 28m 6s	remaining: 1h 16m 22s
269:	learn: 0.7453014	test: 0.7500863

344:	learn: 0.7467426	test: 0.7506235	best: 0.7511224 (316)	total: 38m 10s	remaining: 1h 12m 28s
345:	learn: 0.7468193	test: 0.7504317	best: 0.7511224 (316)	total: 38m 15s	remaining: 1h 12m 19s
346:	learn: 0.7468406	test: 0.7503549	best: 0.7511224 (316)	total: 38m 23s	remaining: 1h 12m 14s
347:	learn: 0.7467980	test: 0.7504317	best: 0.7511224 (316)	total: 38m 32s	remaining: 1h 12m 13s
348:	learn: 0.7468619	test: 0.7504701	best: 0.7511224 (316)	total: 38m 42s	remaining: 1h 12m 12s
349:	learn: 0.7468065	test: 0.7504701	best: 0.7511224 (316)	total: 38m 50s	remaining: 1h 12m 8s
350:	learn: 0.7469515	test: 0.7504317	best: 0.7511224 (316)	total: 39m	remaining: 1h 12m 7s
351:	learn: 0.7468875	test: 0.7506235	best: 0.7511224 (316)	total: 39m 7s	remaining: 1h 12m 1s
352:	learn: 0.7469216	test: 0.7507003	best: 0.7511224 (316)	total: 39m 17s	remaining: 1h 12m
353:	learn: 0.7469216	test: 0.7506235	best: 0.7511224 (316)	total: 39m 24s	remaining: 1h 11m 55s
354:	learn: 0.7469088	test: 0.7505852	best

...

603:	learn: 0.7507930	test: 0.7512375	best: 0.7515828 (572)	total: 1h 8m 14s	remaining: 44m 44s
604:	learn: 0.7508144	test: 0.7512375	best: 0.7515828 (572)	total: 1h 8m 19s	remaining: 44m 36s
605:	learn: 0.7508741	test: 0.7512375	best: 0.7515828 (572)	total: 1h 8m 25s	remaining: 44m 28s
606:	learn: 0.7509295	test: 0.7513526	best: 0.7515828 (572)	total: 1h 8m 29s	remaining: 44m 20s
607:	learn: 0.7509295	test: 0.7511991	best: 0.7515828 (572)	total: 1h 8m 35s	remaining: 44m 13s
608:	learn: 0.7508741	test: 0.7513142	best: 0.7515828 (572)	total: 1h 8m 39s	remaining: 44m 5s
609:	learn: 0.7508741	test: 0.7511991	best: 0.7515828 (572)	total: 1h 8m 44s	remaining: 43m 57s
610:	learn: 0.7509252	test: 0.7513142	best: 0.7515828 (572)	total: 1h 8m 50s	remaining: 43m 49s
611:	learn: 0.7509679	test: 0.7513910	best: 0.7515828 (572)	total: 1h 8m 54s	remaining: 43m 41s
612:	learn: 0.7509892	test: 0.7511991	best: 0.7515828 (572)	total: 1h 8m 58s	remaining: 43m 32s
613:	learn: 0.7509679	test: 0.7511991	bes

689:	learn: 0.7520679	test: 0.7512759	best: 0.7516979 (648)	total: 1h 15m 26s	remaining: 33m 53s
690:	learn: 0.7521105	test: 0.7513142	best: 0.7516979 (648)	total: 1h 15m 32s	remaining: 33m 46s
691:	learn: 0.7520764	test: 0.7512759	best: 0.7516979 (648)	total: 1h 15m 37s	remaining: 33m 39s
692:	learn: 0.7520807	test: 0.7513526	best: 0.7516979 (648)	total: 1h 15m 43s	remaining: 33m 32s
693:	learn: 0.7520935	test: 0.7512759	best: 0.7516979 (648)	total: 1h 15m 48s	remaining: 33m 25s
694:	learn: 0.7520892	test: 0.7513142	best: 0.7516979 (648)	total: 1h 15m 56s	remaining: 33m 19s
695:	learn: 0.7520977	test: 0.7514293	best: 0.7516979 (648)	total: 1h 16m 1s	remaining: 33m 12s
696:	learn: 0.7521361	test: 0.7513910	best: 0.7516979 (648)	total: 1h 16m 7s	remaining: 33m 5s
697:	learn: 0.7521404	test: 0.7513526	best: 0.7516979 (648)	total: 1h 16m 12s	remaining: 32m 58s
698:	learn: 0.7521745	test: 0.7515061	best: 0.7516979 (648)	total: 1h 16m 18s	remaining: 32m 51s
699:	learn: 0.7522001	test: 0.751

774:	learn: 0.7532958	test: 0.7515828	best: 0.7518131 (765)	total: 1h 23m 30s	remaining: 24m 14s
775:	learn: 0.7532788	test: 0.7516212	best: 0.7518131 (765)	total: 1h 23m 35s	remaining: 24m 7s
776:	learn: 0.7533043	test: 0.7517747	best: 0.7518131 (765)	total: 1h 23m 42s	remaining: 24m 1s
777:	learn: 0.7533086	test: 0.7517747	best: 0.7518131 (765)	total: 1h 23m 48s	remaining: 23m 54s
778:	learn: 0.7533299	test: 0.7518131	best: 0.7518131 (765)	total: 1h 23m 54s	remaining: 23m 48s
779:	learn: 0.7532915	test: 0.7518514	best: 0.7518514 (779)	total: 1h 24m 1s	remaining: 23m 42s
780:	learn: 0.7533598	test: 0.7516212	best: 0.7518514 (779)	total: 1h 24m 7s	remaining: 23m 35s
781:	learn: 0.7533555	test: 0.7516979	best: 0.7518514 (779)	total: 1h 24m 12s	remaining: 23m 28s
782:	learn: 0.7534195	test: 0.7514677	best: 0.7518514 (779)	total: 1h 24m 20s	remaining: 23m 22s
783:	learn: 0.7534109	test: 0.7514677	best: 0.7518514 (779)	total: 1h 24m 25s	remaining: 23m 15s
784:	learn: 0.7534109	test: 0.7515

859:	learn: 0.7542722	test: 0.7510456	best: 0.7519665 (818)	total: 1h 32m 38s	remaining: 15m 4s
860:	learn: 0.7542381	test: 0.7509305	best: 0.7519665 (818)	total: 1h 32m 45s	remaining: 14m 58s
861:	learn: 0.7542296	test: 0.7509689	best: 0.7519665 (818)	total: 1h 32m 51s	remaining: 14m 52s
862:	learn: 0.7542296	test: 0.7511607	best: 0.7519665 (818)	total: 1h 32m 56s	remaining: 14m 45s
863:	learn: 0.7541699	test: 0.7512759	best: 0.7519665 (818)	total: 1h 33m 2s	remaining: 14m 38s
864:	learn: 0.7542509	test: 0.7512759	best: 0.7519665 (818)	total: 1h 33m 8s	remaining: 14m 32s
865:	learn: 0.7542509	test: 0.7513142	best: 0.7519665 (818)	total: 1h 33m 13s	remaining: 14m 25s
866:	learn: 0.7542892	test: 0.7513142	best: 0.7519665 (818)	total: 1h 33m 19s	remaining: 14m 19s
867:	learn: 0.7542722	test: 0.7513910	best: 0.7519665 (818)	total: 1h 33m 25s	remaining: 14m 12s
868:	learn: 0.7542594	test: 0.7513526	best: 0.7519665 (818)	total: 1h 33m 30s	remaining: 14m 5s
869:	learn: 0.7542765	test: 0.7513

945:	learn: 0.7553680	test: 0.7521200	best: 0.7521584 (943)	total: 1h 40m 52s	remaining: 5m 45s
946:	learn: 0.7553594	test: 0.7520049	best: 0.7521584 (943)	total: 1h 40m 57s	remaining: 5m 39s
947:	learn: 0.7554021	test: 0.7519282	best: 0.7521584 (943)	total: 1h 41m 3s	remaining: 5m 32s
948:	learn: 0.7553978	test: 0.7518898	best: 0.7521584 (943)	total: 1h 41m 8s	remaining: 5m 26s
949:	learn: 0.7554575	test: 0.7519282	best: 0.7521584 (943)	total: 1h 41m 14s	remaining: 5m 19s
950:	learn: 0.7555001	test: 0.7519282	best: 0.7521584 (943)	total: 1h 41m 21s	remaining: 5m 13s
951:	learn: 0.7554532	test: 0.7519665	best: 0.7521584 (943)	total: 1h 41m 26s	remaining: 5m 6s
952:	learn: 0.7554234	test: 0.7518131	best: 0.7521584 (943)	total: 1h 41m 32s	remaining: 5m
953:	learn: 0.7554021	test: 0.7517747	best: 0.7521584 (943)	total: 1h 41m 37s	remaining: 4m 53s
954:	learn: 0.7554021	test: 0.7517747	best: 0.7521584 (943)	total: 1h 41m 42s	remaining: 4m 47s
955:	learn: 0.7554916	test: 0.7518898	best: 0.7

In [11]:
import pickle
filename = 'pickles/new_cat_boost_pickle.pkl'
pickle.dump(model, open(filename,'wb'))

In [12]:
feature_importances = pd.DataFrame({'features_names':model.feature_names_, 'features_importance': model.feature_importances_})
feature_importances

,features_names,features_importance
0,geo_level_1_id,7.90
1,geo_level_2_id,20.56
2,geo_level_3_id,20.04
3,count_floors_pre_eq,2.07
4,age,7.64
5,area_percentage,3.14
6,height_percentage,2.75
7,land_surface_condition,1.15
8,foundation_type,6.06
9,roof_type,3.12


In [14]:
y_pred=model.predict(X_val)
print(model.score(X_val, y_val))

0.7521583976056175


In [16]:
val_preds = model.predict(X_val)
f1_score(y_val, val_preds, average='micro')

0.7521583976056176

In [17]:
predictions = model.predict(X_test)
predictions = predictions.flatten()
predictions = predictions.astype(int)
predictions

array([3, 2, 2, ..., 2, 2, 1])

In [18]:
output = pd.DataFrame({'building_id':data_test.building_id, 'damage_grade': predictions})
output.to_csv('my_new_cb_submission.csv', index=False)

## Borro features poco importantes y veo que pasa

In [27]:
y = data_train["damage_grade"].values
X = data_train.drop(['damage_grade',
                     'building_id',
                     "has_secondary_use",
                     "has_secondary_use_agriculture",
                     "has_secondary_use_hotel",
                     "has_secondary_use_rental",
                     "has_secondary_use_institution",
                     "has_secondary_use_school",
                     "has_secondary_use_industry",
                     "has_secondary_use_health_post",
                     "has_secondary_use_gov_office",
                     "has_secondary_use_use_police",
                     "has_secondary_use_other"], axis=1)
X_test = data_test.drop(['building_id',
                            "has_secondary_use",
                            "has_secondary_use_agriculture",
                             "has_secondary_use_hotel",
                             "has_secondary_use_rental",
                             "has_secondary_use_institution",
                             "has_secondary_use_school",
                             "has_secondary_use_industry",
                             "has_secondary_use_health_post",
                             "has_secondary_use_gov_office",
                             "has_secondary_use_use_police",
                             "has_secondary_use_other"], axis=1)

In [28]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=1, stratify=y, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

((234540, 28), (26061, 28), (234540,), (26061,), (86868, 28))

In [30]:
cate_features_index = np.where(X.dtypes != float)[0]

In [31]:
from catboost import Pool, CatBoostClassifier, cv
model = CatBoostClassifier(eval_metric='Accuracy', loss_function='MultiClass',use_best_model=True,random_seed=1)
model.fit(X_train,y_train,cat_features=cate_features_index,eval_set=(X_val,y_val))
print(model.score(X_train, y_train))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.120429
0:	learn: 0.6911358	test: 0.6937953	best: 0.6937953 (0)	total: 3.42s	remaining: 56m 51s
1:	learn: 0.6960177	test: 0.6987069	best: 0.6987069 (1)	total: 7.46s	remaining: 1h 2m
2:	learn: 0.6995353	test: 0.7006638	best: 0.7006638 (2)	total: 10.7s	remaining: 59m 18s
3:	learn: 0.6996589	test: 0.7012010	best: 0.7012010 (3)	total: 13.7s	remaining: 56m 48s
4:	learn: 0.7011512	test: 0.7015847	best: 0.7015847 (4)	total: 17.1s	remaining: 56m 42s
5:	learn: 0.7018206	test: 0.7020068	best: 0.7020068 (5)	total: 20.8s	remaining: 57m 27s
6:	learn: 0.7025070	test: 0.7023522	best: 0.7023522 (6)	total: 25.1s	remaining: 59m 25s
7:	learn: 0.7027714	test: 0.7029277	best: 0.7029277 (7)	total: 29.3s	remaining: 1h 37s
8:	learn: 0.7033470	test: 0.7039638	best: 0.7039638 (8)	total: 33.2s	remaining: 1h 59s
9:	learn: 0.7034365	test: 0.7037719	best: 0.7039638 (8)	total: 36.6s	remaining: 1h 26s
10:	learn: 0.7037179	test: 0.7048847	best: 0.7048847 (10)	total: 39.9s	remaining: 59m 47s
11:	l

89:	learn: 0.7376439	test: 0.7448678	best: 0.7449062 (82)	total: 6m 39s	remaining: 1h 7m 14s
90:	learn: 0.7375757	test: 0.7447143	best: 0.7449062 (82)	total: 6m 43s	remaining: 1h 7m 15s
91:	learn: 0.7376482	test: 0.7445992	best: 0.7449062 (82)	total: 6m 48s	remaining: 1h 7m 11s
92:	learn: 0.7377377	test: 0.7449062	best: 0.7449062 (82)	total: 6m 53s	remaining: 1h 7m 9s
93:	learn: 0.7377164	test: 0.7447527	best: 0.7449062 (82)	total: 6m 59s	remaining: 1h 7m 18s
94:	learn: 0.7377164	test: 0.7447527	best: 0.7449062 (82)	total: 7m 4s	remaining: 1h 7m 20s
95:	learn: 0.7378230	test: 0.7448678	best: 0.7449062 (82)	total: 7m 8s	remaining: 1h 7m 15s
96:	learn: 0.7378528	test: 0.7448678	best: 0.7449062 (82)	total: 7m 13s	remaining: 1h 7m 14s
97:	learn: 0.7380276	test: 0.7452899	best: 0.7452899 (97)	total: 7m 18s	remaining: 1h 7m 12s
98:	learn: 0.7381726	test: 0.7454818	best: 0.7454818 (98)	total: 7m 22s	remaining: 1h 7m 6s
99:	learn: 0.7381470	test: 0.7455969	best: 0.7455969 (99)	total: 7m 26s	re

176:	learn: 0.7421080	test: 0.7490503	best: 0.7492038 (174)	total: 13m 27s	remaining: 1h 2m 35s
177:	learn: 0.7420909	test: 0.7493189	best: 0.7493189 (177)	total: 13m 32s	remaining: 1h 2m 29s
178:	learn: 0.7421293	test: 0.7492805	best: 0.7493189 (177)	total: 13m 36s	remaining: 1h 2m 25s
179:	learn: 0.7421847	test: 0.7494340	best: 0.7494340 (179)	total: 13m 41s	remaining: 1h 2m 22s
180:	learn: 0.7422529	test: 0.7495108	best: 0.7495108 (180)	total: 13m 45s	remaining: 1h 2m 14s
181:	learn: 0.7423467	test: 0.7493956	best: 0.7495108 (180)	total: 13m 50s	remaining: 1h 2m 11s
182:	learn: 0.7424107	test: 0.7491654	best: 0.7495108 (180)	total: 13m 55s	remaining: 1h 2m 8s
183:	learn: 0.7424533	test: 0.7494340	best: 0.7495108 (180)	total: 13m 59s	remaining: 1h 2m 4s
184:	learn: 0.7424405	test: 0.7493956	best: 0.7495108 (180)	total: 14m 4s	remaining: 1h 2m 2s
185:	learn: 0.7424661	test: 0.7495491	best: 0.7495491 (185)	total: 14m 9s	remaining: 1h 1m 59s
186:	learn: 0.7425087	test: 0.7495875	best: 0

264:	learn: 0.7446065	test: 0.7493956	best: 0.7499328 (243)	total: 20m 18s	remaining: 56m 20s
265:	learn: 0.7446278	test: 0.7496259	best: 0.7499328 (243)	total: 20m 24s	remaining: 56m 17s
266:	learn: 0.7446448	test: 0.7495108	best: 0.7499328 (243)	total: 20m 28s	remaining: 56m 12s
267:	learn: 0.7447088	test: 0.7494724	best: 0.7499328 (243)	total: 20m 33s	remaining: 56m 8s
268:	learn: 0.7448111	test: 0.7494340	best: 0.7499328 (243)	total: 20m 38s	remaining: 56m 5s
269:	learn: 0.7447258	test: 0.7493189	best: 0.7499328 (243)	total: 20m 43s	remaining: 56m
270:	learn: 0.7446619	test: 0.7494340	best: 0.7499328 (243)	total: 20m 48s	remaining: 55m 58s
271:	learn: 0.7448282	test: 0.7493189	best: 0.7499328 (243)	total: 20m 52s	remaining: 55m 53s
272:	learn: 0.7448708	test: 0.7494340	best: 0.7499328 (243)	total: 20m 57s	remaining: 55m 48s
273:	learn: 0.7449646	test: 0.7494724	best: 0.7499328 (243)	total: 21m 2s	remaining: 55m 45s
274:	learn: 0.7449945	test: 0.7494340	best: 0.7499328 (243)	total: 

...

616:	learn: 0.7504349	test: 0.7509689	best: 0.7518131 (529)	total: 47m 54s	remaining: 29m 44s
617:	learn: 0.7504605	test: 0.7510073	best: 0.7518131 (529)	total: 47m 59s	remaining: 29m 40s
618:	learn: 0.7504690	test: 0.7510073	best: 0.7518131 (529)	total: 48m 4s	remaining: 29m 35s
619:	learn: 0.7505031	test: 0.7509689	best: 0.7518131 (529)	total: 48m 9s	remaining: 29m 31s
620:	learn: 0.7504775	test: 0.7510840	best: 0.7518131 (529)	total: 48m 13s	remaining: 29m 26s
621:	learn: 0.7505031	test: 0.7510073	best: 0.7518131 (529)	total: 48m 19s	remaining: 29m 21s
622:	learn: 0.7504861	test: 0.7510456	best: 0.7518131 (529)	total: 48m 22s	remaining: 29m 16s
623:	learn: 0.7504477	test: 0.7510456	best: 0.7518131 (529)	total: 48m 26s	remaining: 29m 11s
624:	learn: 0.7504733	test: 0.7510456	best: 0.7518131 (529)	total: 48m 32s	remaining: 29m 7s
625:	learn: 0.7504647	test: 0.7510840	best: 0.7518131 (529)	total: 48m 36s	remaining: 29m 2s
626:	learn: 0.7504434	test: 0.7510456	best: 0.7518131 (529)	tota

704:	learn: 0.7516330	test: 0.7514677	best: 0.7518131 (529)	total: 54m 40s	remaining: 22m 52s
705:	learn: 0.7516500	test: 0.7516979	best: 0.7518131 (529)	total: 54m 45s	remaining: 22m 48s
706:	learn: 0.7516245	test: 0.7515061	best: 0.7518131 (529)	total: 54m 50s	remaining: 22m 43s
707:	learn: 0.7516202	test: 0.7515445	best: 0.7518131 (529)	total: 54m 55s	remaining: 22m 39s
708:	learn: 0.7516543	test: 0.7515828	best: 0.7518131 (529)	total: 54m 59s	remaining: 22m 34s
709:	learn: 0.7516628	test: 0.7515445	best: 0.7518131 (529)	total: 55m 4s	remaining: 22m 29s
710:	learn: 0.7516330	test: 0.7515445	best: 0.7518131 (529)	total: 55m 8s	remaining: 22m 24s
711:	learn: 0.7516458	test: 0.7515828	best: 0.7518131 (529)	total: 55m 13s	remaining: 22m 20s
712:	learn: 0.7516543	test: 0.7515061	best: 0.7518131 (529)	total: 55m 18s	remaining: 22m 15s
713:	learn: 0.7516415	test: 0.7515445	best: 0.7518131 (529)	total: 55m 22s	remaining: 22m 10s
714:	learn: 0.7517353	test: 0.7516212	best: 0.7518131 (529)	to

792:	learn: 0.7526008	test: 0.7516979	best: 0.7518898 (785)	total: 1h 1m 38s	remaining: 16m 5s
793:	learn: 0.7526264	test: 0.7516212	best: 0.7518898 (785)	total: 1h 1m 43s	remaining: 16m
794:	learn: 0.7527074	test: 0.7516979	best: 0.7518898 (785)	total: 1h 1m 48s	remaining: 15m 56s
795:	learn: 0.7526989	test: 0.7517363	best: 0.7518898 (785)	total: 1h 1m 52s	remaining: 15m 51s
796:	learn: 0.7527756	test: 0.7517747	best: 0.7518898 (785)	total: 1h 1m 57s	remaining: 15m 46s
797:	learn: 0.7527714	test: 0.7517363	best: 0.7518898 (785)	total: 1h 2m 1s	remaining: 15m 42s
798:	learn: 0.7527884	test: 0.7516979	best: 0.7518898 (785)	total: 1h 2m 5s	remaining: 15m 37s
799:	learn: 0.7528012	test: 0.7519665	best: 0.7519665 (799)	total: 1h 2m 10s	remaining: 15m 32s
800:	learn: 0.7528396	test: 0.7518131	best: 0.7519665 (799)	total: 1h 2m 14s	remaining: 15m 27s
801:	learn: 0.7528353	test: 0.7516979	best: 0.7519665 (799)	total: 1h 2m 18s	remaining: 15m 23s
802:	learn: 0.7527842	test: 0.7517363	best: 0.7

878:	learn: 0.7539482	test: 0.7516979	best: 0.7520049 (872)	total: 1h 8m 20s	remaining: 9m 24s
879:	learn: 0.7539567	test: 0.7516979	best: 0.7520049 (872)	total: 1h 8m 24s	remaining: 9m 19s
880:	learn: 0.7539609	test: 0.7516596	best: 0.7520049 (872)	total: 1h 8m 29s	remaining: 9m 15s
881:	learn: 0.7540121	test: 0.7516212	best: 0.7520049 (872)	total: 1h 8m 34s	remaining: 9m 10s
882:	learn: 0.7539908	test: 0.7516979	best: 0.7520049 (872)	total: 1h 8m 38s	remaining: 9m 5s
883:	learn: 0.7539695	test: 0.7516596	best: 0.7520049 (872)	total: 1h 8m 43s	remaining: 9m 1s
884:	learn: 0.7539780	test: 0.7516212	best: 0.7520049 (872)	total: 1h 8m 48s	remaining: 8m 56s
885:	learn: 0.7539780	test: 0.7519282	best: 0.7520049 (872)	total: 1h 8m 54s	remaining: 8m 51s
886:	learn: 0.7540121	test: 0.7515828	best: 0.7520049 (872)	total: 1h 8m 59s	remaining: 8m 47s
887:	learn: 0.7540590	test: 0.7515828	best: 0.7520049 (872)	total: 1h 9m 4s	remaining: 8m 42s
888:	learn: 0.7540633	test: 0.7516212	best: 0.7520049

964:	learn: 0.7548350	test: 0.7515445	best: 0.7520049 (872)	total: 1h 15m 10s	remaining: 2m 43s
965:	learn: 0.7549160	test: 0.7516212	best: 0.7520049 (872)	total: 1h 15m 15s	remaining: 2m 38s
966:	learn: 0.7548904	test: 0.7516212	best: 0.7520049 (872)	total: 1h 15m 19s	remaining: 2m 34s
967:	learn: 0.7549331	test: 0.7515061	best: 0.7520049 (872)	total: 1h 15m 25s	remaining: 2m 29s
968:	learn: 0.7549459	test: 0.7515061	best: 0.7520049 (872)	total: 1h 15m 29s	remaining: 2m 24s
969:	learn: 0.7548691	test: 0.7514677	best: 0.7520049 (872)	total: 1h 15m 35s	remaining: 2m 20s
970:	learn: 0.7548904	test: 0.7516596	best: 0.7520049 (872)	total: 1h 15m 39s	remaining: 2m 15s
971:	learn: 0.7549459	test: 0.7516212	best: 0.7520049 (872)	total: 1h 15m 44s	remaining: 2m 10s
972:	learn: 0.7549714	test: 0.7516212	best: 0.7520049 (872)	total: 1h 15m 49s	remaining: 2m 6s
973:	learn: 0.7550311	test: 0.7516596	best: 0.7520049 (872)	total: 1h 15m 53s	remaining: 2m 1s
974:	learn: 0.7550098	test: 0.7515828	best

In [32]:
import pickle
filename = 'pickles/menos_features_cat_boost_pickle.pkl'
pickle.dump(model, open(filename,'wb'))

In [33]:
feature_importances = pd.DataFrame({'features_names':model.feature_names_, 'features_importance': model.feature_importances_})
feature_importances

,features_names,features_importance
0,geo_level_1_id,6.87
1,geo_level_2_id,27.33
2,geo_level_3_id,15.09
3,count_floors_pre_eq,1.95
4,age,7.76
5,area_percentage,2.96
6,height_percentage,3.07
7,land_surface_condition,1.27
8,foundation_type,6.70
9,roof_type,3.36


In [34]:
val_preds = model.predict(X_val)
f1_score(y_val, val_preds, average='micro')

0.7520049115536626

In [35]:
predictions = model.predict(X_test)
predictions = predictions.flatten()
predictions = predictions.astype(int)
predictions

array([3, 2, 2, ..., 2, 2, 1])

In [36]:
output = pd.DataFrame({'building_id':data_test.building_id, 'damage_grade': predictions})
output.to_csv('menos_features_cb_submission.csv', index=False)

## Mas lr y mas iteraciones

In [37]:
y = data_train["damage_grade"].values
X = data_train.drop(['damage_grade', 'building_id'], axis=1)
X_test = data_test.drop(['building_id'], axis=1)

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=1, stratify=y, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

((234540, 39), (26061, 39), (234540,), (26061,), (86868, 39))

In [39]:
cate_features_index = np.where(X.dtypes != float)[0]

In [40]:
from catboost import Pool, CatBoostClassifier, cv
modelo = CatBoostClassifier(iterations=15000,eval_metric='Accuracy', loss_function='MultiClass',use_best_model=True,random_seed=1)

In [41]:
modelo.fit(X_train,y_train,cat_features=cate_features_index,eval_set=(X_val,y_val))
print(modelo.score(X_train, y_train))

Learning rate set to 0.042802
0:	learn: 0.6971902	test: 0.6979778	best: 0.6979778 (0)	total: 3.99s	remaining: 16h 37m 51s
1:	learn: 0.6971391	test: 0.6975174	best: 0.6979778 (0)	total: 8.63s	remaining: 17h 57m 59s
2:	learn: 0.6979236	test: 0.6984767	best: 0.6984767 (2)	total: 13.3s	remaining: 18h 31m 41s
3:	learn: 0.6986825	test: 0.6987836	best: 0.6987836 (3)	total: 17.6s	remaining: 18h 16m 38s
4:	learn: 0.6984395	test: 0.6982464	best: 0.6987836 (3)	total: 20.8s	remaining: 17h 19m 39s
5:	learn: 0.6986484	test: 0.6983232	best: 0.6987836 (3)	total: 24.9s	remaining: 17h 17m 58s
6:	learn: 0.6984821	test: 0.6982081	best: 0.6987836 (3)	total: 28.9s	remaining: 17h 12m 56s
7:	learn: 0.6987209	test: 0.6981697	best: 0.6987836 (3)	total: 33s	remaining: 17h 9m 35s
8:	learn: 0.6990449	test: 0.6985918	best: 0.6987836 (3)	total: 36.3s	remaining: 16h 47m 39s
9:	learn: 0.7005457	test: 0.7002417	best: 0.7002417 (9)	total: 40.1s	remaining: 16h 43m 3s
10:	learn: 0.7018717	test: 0.7011243	best: 0.7011243 (

88:	learn: 0.7284855	test: 0.7338936	best: 0.7338936 (88)	total: 6m 41s	remaining: 18h 41m 43s
89:	learn: 0.7285580	test: 0.7342773	best: 0.7342773 (89)	total: 6m 47s	remaining: 18h 45m 59s
90:	learn: 0.7286774	test: 0.7344308	best: 0.7344308 (90)	total: 6m 53s	remaining: 18h 49m 21s
91:	learn: 0.7287286	test: 0.7344308	best: 0.7344308 (90)	total: 6m 58s	remaining: 18h 50m 23s
92:	learn: 0.7287584	test: 0.7345459	best: 0.7345459 (92)	total: 7m 4s	remaining: 18h 54m 2s
93:	learn: 0.7289247	test: 0.7345842	best: 0.7345842 (93)	total: 7m 10s	remaining: 18h 58m 42s
94:	learn: 0.7291422	test: 0.7345459	best: 0.7345842 (93)	total: 7m 16s	remaining: 19h 27s
95:	learn: 0.7292146	test: 0.7346610	best: 0.7346610 (95)	total: 7m 21s	remaining: 19h 3m 2s
96:	learn: 0.7293468	test: 0.7350063	best: 0.7350063 (96)	total: 7m 27s	remaining: 19h 5m 59s
97:	learn: 0.7295301	test: 0.7348145	best: 0.7350063 (96)	total: 7m 32s	remaining: 19h 6m 25s
98:	learn: 0.7295515	test: 0.7350063	best: 0.7350063 (96)	to

173:	learn: 0.7342202	test: 0.7415295	best: 0.7417981 (170)	total: 14m 18s	remaining: 20h 18m 36s
174:	learn: 0.7341946	test: 0.7414911	best: 0.7417981 (170)	total: 14m 22s	remaining: 20h 17m 49s
175:	learn: 0.7343182	test: 0.7419132	best: 0.7419132 (175)	total: 14m 27s	remaining: 20h 17m 30s
176:	learn: 0.7343097	test: 0.7418365	best: 0.7419132 (175)	total: 14m 32s	remaining: 20h 17m 54s
177:	learn: 0.7343438	test: 0.7416446	best: 0.7419132 (175)	total: 14m 37s	remaining: 20h 17m 36s
178:	learn: 0.7343822	test: 0.7416830	best: 0.7419132 (175)	total: 14m 41s	remaining: 20h 16m 27s
179:	learn: 0.7344419	test: 0.7415679	best: 0.7419132 (175)	total: 14m 46s	remaining: 20h 16m 22s
180:	learn: 0.7346210	test: 0.7416830	best: 0.7419132 (175)	total: 14m 52s	remaining: 20h 17m 29s
181:	learn: 0.7346423	test: 0.7417597	best: 0.7419132 (175)	total: 14m 57s	remaining: 20h 17m 29s
182:	learn: 0.7346593	test: 0.7418748	best: 0.7419132 (175)	total: 15m 1s	remaining: 20h 16m 44s
183:	learn: 0.7347276

257:	learn: 0.7377548	test: 0.7442539	best: 0.7445608 (252)	total: 21m 23s	remaining: 20h 22m 40s
258:	learn: 0.7378059	test: 0.7442922	best: 0.7445608 (252)	total: 21m 28s	remaining: 20h 22m 35s
259:	learn: 0.7377931	test: 0.7440236	best: 0.7445608 (252)	total: 21m 34s	remaining: 20h 22m 44s
260:	learn: 0.7378315	test: 0.7440620	best: 0.7445608 (252)	total: 21m 38s	remaining: 20h 22m 33s
261:	learn: 0.7378571	test: 0.7439853	best: 0.7445608 (252)	total: 21m 43s	remaining: 20h 22m 14s
262:	learn: 0.7378144	test: 0.7437550	best: 0.7445608 (252)	total: 21m 48s	remaining: 20h 22m 19s
263:	learn: 0.7377505	test: 0.7437167	best: 0.7445608 (252)	total: 21m 53s	remaining: 20h 21m 37s
264:	learn: 0.7378315	test: 0.7440620	best: 0.7445608 (252)	total: 21m 57s	remaining: 20h 21m 17s
265:	learn: 0.7378230	test: 0.7439469	best: 0.7445608 (252)	total: 22m 2s	remaining: 20h 21m 18s
266:	learn: 0.7378699	test: 0.7439085	best: 0.7445608 (252)	total: 22m 8s	remaining: 20h 21m 35s
267:	learn: 0.7379040	

341:	learn: 0.7397203	test: 0.7456352	best: 0.7460190 (332)	total: 28m 55s	remaining: 20h 39m 52s
342:	learn: 0.7396691	test: 0.7457504	best: 0.7460190 (332)	total: 29m 2s	remaining: 20h 40m 45s
343:	learn: 0.7397331	test: 0.7457887	best: 0.7460190 (332)	total: 29m 8s	remaining: 20h 41m 19s
344:	learn: 0.7398098	test: 0.7457887	best: 0.7460190 (332)	total: 29m 14s	remaining: 20h 42m 16s
345:	learn: 0.7398312	test: 0.7459038	best: 0.7460190 (332)	total: 29m 19s	remaining: 20h 42m 5s
346:	learn: 0.7398695	test: 0.7458655	best: 0.7460190 (332)	total: 29m 26s	remaining: 20h 43m 5s
347:	learn: 0.7398994	test: 0.7459806	best: 0.7460190 (332)	total: 29m 31s	remaining: 20h 43m 26s
348:	learn: 0.7399335	test: 0.7459806	best: 0.7460190 (332)	total: 29m 37s	remaining: 20h 43m 19s
349:	learn: 0.7399548	test: 0.7460957	best: 0.7460957 (349)	total: 29m 42s	remaining: 20h 43m 35s
350:	learn: 0.7400443	test: 0.7457887	best: 0.7460957 (349)	total: 29m 48s	remaining: 20h 44m 8s
351:	learn: 0.7400998	tes

...

14630:	learn: 0.8067409	test: 0.7544991	best: 0.7549595 (11391)	total: 23h 59m 15s	remaining: 36m 17s
14631:	learn: 0.8067494	test: 0.7544991	best: 0.7549595 (11391)	total: 23h 59m 21s	remaining: 36m 12s
14632:	learn: 0.8067451	test: 0.7544223	best: 0.7549595 (11391)	total: 23h 59m 29s	remaining: 36m 6s
14633:	learn: 0.8067835	test: 0.7543839	best: 0.7549595 (11391)	total: 23h 59m 34s	remaining: 36m
14634:	learn: 0.8067750	test: 0.7544223	best: 0.7549595 (11391)	total: 23h 59m 40s	remaining: 35m 54s
14635:	learn: 0.8068048	test: 0.7544607	best: 0.7549595 (11391)	total: 23h 59m 47s	remaining: 35m 48s
14636:	learn: 0.8067664	test: 0.7544223	best: 0.7549595 (11391)	total: 23h 59m 53s	remaining: 35m 42s
14637:	learn: 0.8067622	test: 0.7544607	best: 0.7549595 (11391)	total: 23h 59m 59s	remaining: 35m 36s
14638:	learn: 0.8067920	test: 0.7544607	best: 0.7549595 (11391)	total: 1d 5s	remaining: 35m 30s
14639:	learn: 0.8067920	test: 0.7544607	best: 0.7549595 (11391)	total: 1d 13s	remaining: 35m 

14713:	learn: 0.8070819	test: 0.7544607	best: 0.7549595 (11391)	total: 1d 9m 27s	remaining: 28m 10s
14714:	learn: 0.8070649	test: 0.7544991	best: 0.7549595 (11391)	total: 1d 9m 34s	remaining: 28m 4s
14715:	learn: 0.8070436	test: 0.7545374	best: 0.7549595 (11391)	total: 1d 9m 43s	remaining: 27m 58s
14716:	learn: 0.8070521	test: 0.7544607	best: 0.7549595 (11391)	total: 1d 9m 51s	remaining: 27m 52s
14717:	learn: 0.8070180	test: 0.7544991	best: 0.7549595 (11391)	total: 1d 9m 58s	remaining: 27m 46s
14718:	learn: 0.8070478	test: 0.7545374	best: 0.7549595 (11391)	total: 1d 10m 6s	remaining: 27m 41s
14719:	learn: 0.8070393	test: 0.7545374	best: 0.7549595 (11391)	total: 1d 10m 12s	remaining: 27m 35s
14720:	learn: 0.8070692	test: 0.7545758	best: 0.7549595 (11391)	total: 1d 10m 20s	remaining: 27m 29s
14721:	learn: 0.8071033	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 10m 27s	remaining: 27m 23s
14722:	learn: 0.8071118	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 10m 34s	remaining: 27m 1

14795:	learn: 0.8074358	test: 0.7545758	best: 0.7549595 (11391)	total: 1d 19m 42s	remaining: 20m 7s
14796:	learn: 0.8074529	test: 0.7546525	best: 0.7549595 (11391)	total: 1d 19m 50s	remaining: 20m 1s
14797:	learn: 0.8074358	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 19m 57s	remaining: 19m 55s
14798:	learn: 0.8074358	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 20m 5s	remaining: 19m 49s
14799:	learn: 0.8074486	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 20m 13s	remaining: 19m 43s
14800:	learn: 0.8074316	test: 0.7545758	best: 0.7549595 (11391)	total: 1d 20m 21s	remaining: 19m 38s
14801:	learn: 0.8074742	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 20m 30s	remaining: 19m 32s
14802:	learn: 0.8074444	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 20m 38s	remaining: 19m 26s
14803:	learn: 0.8074657	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 20m 44s	remaining: 19m 20s
14804:	learn: 0.8074614	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 20m 51s	remaining: 1

14877:	learn: 0.8077556	test: 0.7546525	best: 0.7549595 (11391)	total: 1d 29m 43s	remaining: 12m 3s
14878:	learn: 0.8077897	test: 0.7546525	best: 0.7549595 (11391)	total: 1d 29m 52s	remaining: 11m 57s
14879:	learn: 0.8077897	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 29m 59s	remaining: 11m 51s
14880:	learn: 0.8077684	test: 0.7546525	best: 0.7549595 (11391)	total: 1d 30m 7s	remaining: 11m 45s
14881:	learn: 0.8077982	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 30m 13s	remaining: 11m 39s
14882:	learn: 0.8077897	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 30m 20s	remaining: 11m 33s
14883:	learn: 0.8077855	test: 0.7545374	best: 0.7549595 (11391)	total: 1d 30m 28s	remaining: 11m 27s
14884:	learn: 0.8077982	test: 0.7546142	best: 0.7549595 (11391)	total: 1d 30m 35s	remaining: 11m 21s
14885:	learn: 0.8077940	test: 0.7546525	best: 0.7549595 (11391)	total: 1d 30m 43s	remaining: 11m 15s
14886:	learn: 0.8078153	test: 0.7546525	best: 0.7549595 (11391)	total: 1d 30m 50s	remaining: 

14960:	learn: 0.8080754	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 40m 3s	remaining: 3m 51s
14961:	learn: 0.8080754	test: 0.7547293	best: 0.7549595 (11391)	total: 1d 40m 9s	remaining: 3m 45s
14962:	learn: 0.8080882	test: 0.7547677	best: 0.7549595 (11391)	total: 1d 40m 15s	remaining: 3m 39s
14963:	learn: 0.8080711	test: 0.7546525	best: 0.7549595 (11391)	total: 1d 40m 22s	remaining: 3m 33s
14964:	learn: 0.8080583	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 40m 28s	remaining: 3m 27s
14965:	learn: 0.8081010	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 40m 35s	remaining: 3m 21s
14966:	learn: 0.8081223	test: 0.7547293	best: 0.7549595 (11391)	total: 1d 40m 42s	remaining: 3m 15s
14967:	learn: 0.8081223	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 40m 49s	remaining: 3m 9s
14968:	learn: 0.8081052	test: 0.7547677	best: 0.7549595 (11391)	total: 1d 40m 55s	remaining: 3m 4s
14969:	learn: 0.8080882	test: 0.7546909	best: 0.7549595 (11391)	total: 1d 41m 3s	remaining: 2m 58s
14970

In [42]:
filename = 'pickles/new_cat_boost_15k_pickle.pkl'
pickle.dump(modelo, open(filename,'wb'))

In [43]:
feature_importances = pd.DataFrame({'features_names':modelo.feature_names_, 'features_importance': modelo.feature_importances_})
feature_importances

,features_names,features_importance
0,geo_level_1_id,7.40
1,geo_level_2_id,18.83
2,geo_level_3_id,10.65
3,count_floors_pre_eq,3.25
4,age,8.02
5,area_percentage,5.66
6,height_percentage,5.19
7,land_surface_condition,2.41
8,foundation_type,3.68
9,roof_type,3.48


In [44]:
val_preds = modelo.predict(X_val)
f1_score(y_val, val_preds, average='micro')

0.754959518053797

In [45]:
predictions = modelo.predict(X_test)
predictions = predictions.flatten()
predictions = predictions.astype(int)
predictions

array([3, 2, 2, ..., 2, 2, 1])

In [46]:
output = pd.DataFrame({'building_id':data_test.building_id, 'damage_grade': predictions})
output.to_csv('my_new_15k_submission.csv', index=False)

## GridSearch

In [19]:
from catboost import Pool, CatBoostClassifier, cv
modelo = CatBoostClassifier(iterations = 10,eval_metric='Accuracy', loss_function='MultiClass',random_seed=1)

In [ ]:
grid = {'learning_rate': [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.15],
        'depth': [None,4, 6, 10,12,15],
        'l2_leaf_reg': [None,1, 3, 5, 7, 9,12,15,20]}

grid_search_result = modelo.grid_search(grid, 
                                       X=train_data,
                                       cv = 10,
                                       plot=True)